In [1]:
import pandas as pd
from sqlalchemy import create_engine
import re

In [2]:
def clean_column_names(column_name: str) -> str:
    cleaned_name = column_name.replace(' ', '_')
    cleaned_name = re.sub(r'[^\w_]', '', cleaned_name)
    return cleaned_name

In [3]:
username = 'logan'
password = 'password'
host = 'localhost'
port = '3306'
database = 'neeti_sql'

In [4]:
csv_file_path = 'BRS Production Report-3.csv'

In [5]:
engine = create_engine(f'mysql+mysqlconnector://{username}:{password}@{host}:{port}/{database}')

In [6]:
data = pd.read_csv(csv_file_path)

In [7]:
data.columns = [clean_column_names(col) for col in data.columns]

In [8]:
data.head()

,Added_User,Added_Time,Date,Select_Shift,Select_Plant,Bittern_GPL,Be,Ca2,pH,Bittern_Temp,...,Electrical_Related_Shutdown_Hours,Instrument_Related_Hours,Utility_Related_Hours,Operational_Shutdown_Hours,Planned_Shutdown_Hours,RM_Shortage_Shutdown_Hours,Raw_water_Shortage_Hours,Resources_Shutdown_Hours,Running_Hours,Shutdown_Hours
0,sonu.rai_agrocel1,16-Jan-2024 22:17:51,16-Jan-2024,First Shift,S3,0.324,26.0,0.0,2.0,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.5
1,sonu.rai_agrocel1,16-Jan-2024 22:18:36,16-Jan-2024,First Shift,S4,0.324,26.0,0.0,2.0,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.5
2,sonu.rai_agrocel1,16-Jan-2024 22:19:25,16-Jan-2024,First Shift,S5,0.324,26.0,0.0,2.0,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.5
3,sonu.rai_agrocel1,16-Jan-2024 22:20:51,16-Jan-2024,First Shift,S6,0.324,26.0,0.0,2.0,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.5
4,sonu.rai_agrocel1,16-Jan-2024 22:21:26,16-Jan-2024,First Shift,S7,0.324,26.0,0.0,2.0,19.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.5


In [9]:
table_name = 'brs_production_report'
data.to_sql(table_name, engine, if_exists='replace', index=False, method='multi', chunksize=100)

22484

In [10]:
import mysql.connector

import config

cnx = mysql.connector.connect(user=config.MYSQL_USER, password=config.MYSQL_PASSWORD,
                              host=config.MYSQL_HOST, port=config.MYSQL_PORT, database=config.MYSQL_DB_NAME)

In [11]:
def get_column_names(cnx: mysql.connector.MySQLConnection, table_name: str) -> list[str]:
    """Return a list of column names."""
    cursor = cnx.cursor()
    column_names = []
    cursor.execute(
        f"SELECT * FROM `INFORMATION_SCHEMA`.`COLUMNS` WHERE `TABLE_SCHEMA`='{config.MYSQL_DB_NAME}' AND `TABLE_NAME`='{table_name}';")
    for col in cursor:
        column_names.append([col[3], col[7]])
    cursor.close()
    return column_names

In [12]:
data_definations = {
    "tables": [
        {
            "name": table_name,
            "description": "This table stores detailed information about different parameters of BRS production.",
            "columns": []
        }
    ]
}

In [13]:
columns = get_column_names(cnx, table_name)

In [14]:
for column in columns:
    data_definations['tables'][0]['columns'].append(
        {
            "name": str(column[0]),
            "type": str(column[1]),
            "description": ""
        }
    )

In [15]:
import json

with open('data_definations.json', 'w') as file:
    file.write(json.dumps(data_definations, indent=2))